# Step 0: Getting everything ready

Installing the correct packages

In [ ]:
import sys
!{sys.executable} -m pip uninstall tensorflow

In [72]:
!{sys.executable} -m pip install -r requirements.txt

    100% |████████████████████████████████| 583kB 1.5MB/s eta 0:00:01
  Found existing installation: setuptools 39.0.1
    Uninstalling setuptools-39.0.1:
      Successfully uninstalled setuptools-39.0.1


### Now, we need to create the folders necessary for our work 

In [81]:
import os

general_videos_folder = "./videos/"                # where everything related to the videos is
inputs_subfolder = "inputs/"                       # where all videos are
frames_subfolder = "frames/"                       # where the frames of the original video are
aligned_frames_subfolder = "aligned_frames/"       # where the aligned frames of the original video are
generated_images_subfolder = "generated_images/"   # where the generated images of the aligned frames are
latent_representations_subfolder = "latent_repr/"  # where the latent_representations of the aligned frames are
processed_aligned_frames_subfolder = "processed_aligned_frames/" #where the changed aligned frames are 
processed_frames_subfolder = "processed_frames/"   # where the final processed frames are 
no_sound_videos_subfolder = "no_sound_videos/"     # where the output videos from the processed frames are
audio_subfolder = "audios/"                        # where the mp3 of all videos are
final_videos_subfolder = "final_videos/"           # where the final videos with sound are



os.makedirs(general_videos_folder, exist_ok = True)
os.makedirs(f"{general_videos_folder}{inputs_subfolder}", exist_ok = True)
os.makedirs(f"{general_videos_folder}{frames_subfolder}", exist_ok = True)
os.makedirs(f"{general_videos_folder}{aligned_frames_subfolder}", exist_ok = True)
os.makedirs(f"{general_videos_folder}{generated_images_subfolder}", exist_ok = True)
os.makedirs(f"{general_videos_folder}{latent_representations_subfolder}", exist_ok=True)
os.makedirs(f"{general_videos_folder}{processed_aligned_frames_subfolder}", exist_ok=True)
os.makedirs(f"{general_videos_folder}{processed_frames_subfolder}", exist_ok = True)
os.makedirs(f"{general_videos_folder}{no_sound_videos_subfolder}", exist_ok = True)
os.makedirs(f"{general_videos_folder}{audio_subfolder}", exist_ok = True)
os.makedirs(f"{general_videos_folder}{final_videos_subfolder}", exist_ok = True)

This will be the name of the video we are trying to process. This assumes that there is a video called `news_base.mp4` in our `videos/inputs` folder

In [100]:
filename = "news_base"

#### You can take a look at the original video:

In [106]:
from IPython.display import HTML

HTML(f"""
<div align="middle">
<video width="80%" controls>
      <source src="{general_videos_folder}{inputs_subfolder}{filename}.mp4" type="video/mp4">
</video></div>""")

# Step 1: Break the video into multiple frames

For this we use the cv2 library. This code is based on the [following link](https://www.geeksforgeeks.org/python-program-extract-frames-using-opencv/)

In [5]:
import cv2

def video_to_frame(filename):
    os.makedirs(f"{general_videos_folder}{frames_subfolder}{filename}/", exist_ok = True)
    get_frame_path = lambda part: f"{general_videos_folder}{frames_subfolder}{filename}/frame-{part}.jpg" 
    vid_obj = cv2.VideoCapture(f"{general_videos_folder}{inputs_subfolder}{filename}.mp4")
    count = 0
    print(f"Total # of frames according to code {int(vid_obj.get(cv2.CAP_PROP_FRAME_COUNT))}")
    fps = vid_obj.get(cv2.CAP_PROP_FPS)
    print(f"Frames per second = {fps}")
    print("Now reading the frames... [This might take a while]")
    while True: 
        success, image = vid_obj.read() 
        if not success:
            break
        # Saves the frames with frame-count 
        cv2.imwrite(get_frame_path(count), image) 
        count += 1
    print(f"Total # frames accroding to loop = {count}")
    print(f"Succesfully wrote all the frames in the folder {general_videos_folder}{frames_subfolder}{filename}/")
    return fps

## Apply function to our video

This will save the fps (frames per second) of the video in the variable `video_fps`. We will need this value when we are trying to combine back our processed frames into a new video. See [**Step 3**](#Now-let's-convert-our-frames-to-a-video!) for more on this.

In [6]:
video_fps = video_to_frame(filename) #this function return the fps of such video
print(f"FPS returned from function: {video_fps}")

Total # of frames according to code 150
Frames per second = 29.97002997002997
Now reading the frames... [This might take a while]
Total # frames accroding to loop = 150
Succesfully wrote all the frames in the folder ./videos/frames/news_base/
FPS returned from function: 29.97002997002997


# Step 2: Updating every frame

This is where the `stylegan-encoder` code will be most useful

In [109]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


Making sure that the tensorflow version is the correct (1.14.0)

In [110]:
print(tf.__version__)

1.14.0


### Importing necessary libraries

In [10]:
import os
import pickle
import PIL.Image
import numpy as np
import dnnlib
import dnnlib.tflib as tflib
import config
from encoder.generator_model import Generator
import matplotlib.pyplot as plt

/u/ralcanta/MIT_SUMMER_2020/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/u/ralcanta/MIT_SUMMER_2020/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/u/ralcanta/MIT_SUMMER_2020/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/u/ralcanta/MIT_SUMMER_2020/env/lib/python3.6/site-pack

## Step 2.1:  Getting the aligned images out of every frame

In [31]:
import align_images

FRAMES_FOLDER = f"{general_videos_folder}{frames_subfolder}{filename}"
ALIGNED_FOLDER = f"{general_videos_folder}{aligned_frames_subfolder}{filename}"
ALL_ALIGNED_INFO = align_images.align(FRAMES_FOLDER, ALIGNED_FOLDER)  #returns important info about the coordinates of the face in every frame

Using TensorFlow backend.


paths ./videos/frames/news_base and ./videos/aligned_frames/news_base being created if they dont exist
adding ('frame-77', 1, array([[482.54631087,  24.69816544],
       [473.74816544, 349.90368913],
       [798.95368913, 358.70183456],
       [807.75183456,  33.49631087]]))
adding ('frame-107', 1, array([[500.09270371,  33.62910063],
       [476.02910063, 350.05729629],
       [792.45729629, 374.12089937],
       [816.52089937,  57.69270371]]))
adding ('frame-64', 1, array([[474.55004416,  22.92976637],
       [462.70476637, 353.77495584],
       [793.54995584, 365.62023363],
       [805.39523363,  34.77504416]]))
adding ('frame-131', 1, array([[448.53250918,  35.22721721],
       [450.70221721, 379.04249082],
       [794.51749082, 376.87278279],
       [792.34778279,  33.05750918]]))
adding ('frame-131', 2, array([[166.58902772, 291.47136991],
       [156.44636991, 388.93597228],
       [253.91097228, 399.07863009],
       [264.05363009, 301.61402772]]))
adding ('frame-115', 1, array

adding ('frame-65', 1, array([[473.82127118,  21.26365898],
       [461.16365898, 355.12872882],
       [795.02872882, 367.78634102],
       [807.68634102,  33.92127118]]))
adding ('frame-3', 1, array([[471.19210536,  21.18539069],
       [450.68539069, 355.90789464],
       [785.40789464, 376.41460931],
       [805.91460931,  41.69210536]]))
adding ('frame-143', 1, array([[468.6890459 ,  31.29198196],
       [463.84198196, 357.6609541 ],
       [790.2109541 , 362.50801804],
       [795.05801804,  36.1390459 ]]))
adding ('frame-85', 1, array([[480.25388452,  31.71454631],
       [480.08954631, 365.32111548],
       [813.69611548, 365.48545369],
       [813.86045369,  31.87888452]]))
adding ('frame-63', 1, array([[476.73784599,  23.65240572],
       [462.85240572, 351.51215401],
       [790.71215401, 365.39759428],
       [804.59759428,  37.53784599]]))
adding ('frame-82', 1, array([[475.4101754,  24.3268294],
       [475.0768294, 366.3398246],
       [817.0898246, 366.6731706],
       

adding ('frame-43', 1, array([[486.11137933,  29.85542715],
       [466.23042715, 348.11362067],
       [784.48862067, 367.99457285],
       [804.36957285,  49.73637933]]))
adding ('frame-110', 1, array([[492.54268905,  35.20010524],
       [472.70010524, 353.65731095],
       [791.15731095, 373.49989476],
       [810.99989476,  55.04268905]]))
adding ('frame-110', 2, array([[163.10132812, 277.65251514],
       [141.50251514, 396.19867188],
       [260.04867188, 417.79748486],
       [281.64748486, 299.25132812]]))
adding ('frame-96', 1, array([[486.36770075,  22.71491919],
       [473.63991919, 359.25729925],
       [810.18229925, 371.98508081],
       [822.91008081,  35.44270075]]))
adding ('frame-108', 1, array([[503.38916576,  45.62433153],
       [483.02433153, 348.51083424],
       [785.91083424, 368.87566847],
       [806.27566847,  65.98916576]]))
adding ('frame-13', 1, array([[472.65697913,  18.73932611],
       [450.26432611, 353.46802087],
       [784.99302087, 375.86067389]

adding ('frame-111', 2, array([[164.98219519, 277.09775006],
       [140.24775006, 394.21780481],
       [257.36780481, 418.95224994],
       [282.10224994, 301.83219519]]))
adding ('frame-8', 1, array([[472.1100257 ,  20.28040219],
       [449.80540219, 355.0149743 ],
       [784.5399743 , 377.31959781],
       [806.84459781,  42.5850257 ]]))
adding ('frame-83', 1, array([[478.31213514,  28.78315142],
       [478.80815142, 365.41286486],
       [815.43786486, 364.91684858],
       [814.94184858,  28.28713514]]))
adding ('frame-91', 1, array([[482.63218504,  27.62068234],
       [477.69568234, 361.49281496],
       [811.56781496, 366.42931766],
       [816.50431766,  32.55718504]]))
adding ('frame-92', 1, array([[481.22957597,  23.96582775],
       [474.61582775, 362.92042403],
       [813.57042403, 369.53417225],
       [820.18417225,  30.57957597]]))
adding ('frame-129', 1, array([[446.27443684,  32.9078897 ],
       [448.6078897 , 377.92556316],
       [793.62556316, 375.5921103 ],


In [82]:
ALL_ALIGNED_INFO

[('frame-77',
  1,
  array([[482.54631087,  24.69816544],
         [473.74816544, 349.90368913],
         [798.95368913, 358.70183456],
         [807.75183456,  33.49631087]])),
 ('frame-107',
  1,
  array([[500.09270371,  33.62910063],
         [476.02910063, 350.05729629],
         [792.45729629, 374.12089937],
         [816.52089937,  57.69270371]])),
 ('frame-64',
  1,
  array([[474.55004416,  22.92976637],
         [462.70476637, 353.77495584],
         [793.54995584, 365.62023363],
         [805.39523363,  34.77504416]])),
 ('frame-131',
  1,
  array([[448.53250918,  35.22721721],
         [450.70221721, 379.04249082],
         [794.51749082, 376.87278279],
         [792.34778279,  33.05750918]])),
 ('frame-131',
  2,
  array([[166.58902772, 291.47136991],
         [156.44636991, 388.93597228],
         [253.91097228, 399.07863009],
         [264.05363009, 301.61402772]])),
 ('frame-115',
  1,
  array([[470.82021879,  26.67712108],
         [458.27712108, 361.37978121],
         

## Step 2.2: Generating the latent vectors from the aligned images

The latent vectors will be useful to change the `affect` in every frame (see next step)

*WARNING* : This will take quite a while (**read: hours**), so I'd suggest watching [netflix](https://www.netflix.com/) in the meantime

In [6]:
import encode_images
ALIGNED_FOLDER = f"{general_videos_folder}{aligned_frames_subfolder}{filename}"
GENERATED_FOLDER = f"{general_videos_folder}{generated_images_subfolder}{filename}"
LATENT_REPR_FOLDER = f"{general_videos_folder}{latent_representations_subfolder}{filename}"

encode_images.encode(ALIGNED_FOLDER, GENERATED_FOLDER, LATENT_REPR_FOLDER, lazy_update = True)

Using TensorFlow backend.




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





 19%|█▉        | 32/169 [00:00<00:00, 319.29it/s]

lazy update = True and all the images/.npy files for ['frame-64_01'] already exist, so we are skipping this
lazy update = True and all the images/.npy files for ['frame-50_01'] already exist, so we are skipping this
lazy update = True and all the images/.npy files for ['frame-103_01'] already exist, so we are skipping this
lazy update = True and all the images/.npy files for ['frame-110_01'] already exist, so we are skipping this
lazy update = True and all the images/.npy files for ['frame-146_01'] already exist, so we are skipping this
lazy update = True and all the images/.npy files for ['frame-72_01'] already exist, so we are skipping this
lazy update = True and all the images/.npy files for ['frame-0_01'] already exist, so we are skipping this
lazy update = True and all the images/.npy files for ['frame-134_01'] already exist, so we are skipping this
lazy update = True and all the images/.npy files for ['frame-105_01'] already exist, so we are skipping this
lazy update = True and a

 38%|███▊      | 64/169 [00:00<00:00, 317.91it/s]

...

lazy update = True and all the images/.npy files for ['frame-132_02'] already exist, so we are skipping this
lazy update = True and all the images/.npy files for ['frame-104_01'] already exist, so we are skipping this
lazy update = True and all the images/.npy files for ['frame-12_01'] already exist, so we are skipping this
lazy update = True and all the images/.npy files for ['frame-16_01'] already exist, so we are skipping this
lazy update = True and all the images/.npy files for ['frame-138_01'] already exist, so we are skipping this
lazy update = True and all the images/.npy files for ['frame-149_01'] already exist, so we are skipping this
lazy update = True and all the images/.npy files for ['frame-5_01'] already exist, so we are skipping this
lazy update = True and all the images/.npy files for ['frame-57_01'] already exist, so we are skipping this
lazy update = True and all the images/.npy files for ['frame-128_01'] already exist, so we are skipping this
lazy update = True and a

 78%|███████▊  | 131/169 [00:00<00:00, 324.03it/s]

lazy update = True and all the images/.npy files for ['frame-38_01'] already exist, so we are skipping this
lazy update = True and all the images/.npy files for ['frame-69_01'] already exist, so we are skipping this
lazy update = True and all the images/.npy files for ['frame-20_01'] already exist, so we are skipping this
lazy update = True and all the images/.npy files for ['frame-131_01'] already exist, so we are skipping this
lazy update = True and all the images/.npy files for ['frame-7_01'] already exist, so we are skipping this
lazy update = True and all the images/.npy files for ['frame-144_01'] already exist, so we are skipping this
lazy update = True and all the images/.npy files for ['frame-53_01'] already exist, so we are skipping this
lazy update = True and all the images/.npy files for ['frame-111_01'] already exist, so we are skipping this
lazy update = True and all the images/.npy files for ['frame-106_01'] already exist, so we are skipping this
lazy update = True and al

100%|██████████| 169/169 [00:00<00:00, 324.70it/s]

lazy update = True and all the images/.npy files for ['frame-17_01'] already exist, so we are skipping this
lazy update = True and all the images/.npy files for ['frame-126_02'] already exist, so we are skipping this
lazy update = True and all the images/.npy files for ['frame-140_01'] already exist, so we are skipping this
lazy update = True and all the images/.npy files for ['frame-92_01'] already exist, so we are skipping this
lazy update = True and all the images/.npy files for ['frame-11_01'] already exist, so we are skipping this
lazy update = True and all the images/.npy files for ['frame-37_01'] already exist, so we are skipping this
lazy update = True and all the images/.npy files for ['frame-127_02'] already exist, so we are skipping this
lazy update = True and all the images/.npy files for ['frame-113_02'] already exist, so we are skipping this
lazy update = True and all the images/.npy files for ['frame-93_01'] already exist, so we are skipping this
lazy update = True and a

## Step 2.3: Changing the affect of the *aligned* frames, and use this to change the affect of the *original* frames

If you want to see some picture, then you can use `matplotlib`'s `pyplot`

In [35]:
%matplotlib inline
#The line above is necesary to show Matplotlib's plots inside a Jupyter Notebook

from matplotlib import pyplot as plt

Now importing other libraries that will be necessary

In [ ]:
from itertools import chain
from wand.color import Color 
from wand.image import Image as WandImage
from wand.display import display
import cv2
import numpy as np
import os

Now we will define some functions to save the *changed* aligned frames

In [12]:
#URL_FFHQ = 'https://drive.google.com/uc?id=1MEGjdvVpUsu1jB4zrXZN7Y4kBBOzizDQ'
URL_FFHQ = 'https://drive.google.com/uc?id=1PrjiBIXQ-tEa3l3vAtBR7FLi-n9WnVB8'
tflib.init_tf()
with dnnlib.util.open_url(URL_FFHQ, cache_dir=config.cache_dir) as f:
    generator_network, discriminator_network, Gs_network = pickle.load(f)
generator = Generator(Gs_network, batch_size=1, randomize_noise=False)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [52]:
'''
Generates an image from a particular latent vector
'''
def generate_image(latent_vector):
    latent_vector = latent_vector.reshape((1, 18, 512))
    generator.set_dlatents(latent_vector)
    img_array = generator.generate_images()[0]
    img = PIL.Image.fromarray(img_array, 'RGB')
    return img.resize((256, 256))
'''
Change the emotion by a certain coefficient, and save the resulting image
'''
def save_modified_image(latent_vector, direction, coeff, output):
    new_latent_vector = latent_vector.copy()
    new_latent_vector[:8] = (latent_vector + coeff*direction)[:8]
    img = generate_image(new_latent_vector)
    img = img.save(output) 
#     plt.imshow(new_latent_vector)
#     plt.show()

Function that *blends* a picture into another. This code is based on the [following link](https://www.learnopencv.com/seamless-cloning-using-opencv-python-cpp/)

In [85]:
'''
Note: original_frame_path is a *path*, but cropped_face is a numpy array representing the cropped face
'''
def smooth_image(original_frame_path, cropped_face, face_dimension, face_center, output_path):
    print("Template...")
    print(cropped_face.shape, type(cropped_face),  cropped_face.dtype)
#   plt.imshow(cropped_face)
#   plt.show()
    dst = cv2.imread(original_frame_path)
    print("Template info...")
    print(dst.shape, type(dst), dst.dtype)
    ## Now, we create a mask for the face
    src_mask = np.zeros(cropped_face.shape, cropped_face.dtype)
    face_w, face_h = face_dimension
    poly = np.array([[0,0], [face_w, 0], [face_w, face_h], [0, face_h]], np.int32)
    cv2.fillPoly(src_mask, [poly], (255, 255, 255))

    ## Now we are ready to call seamlessClone 
    output = cv2.seamlessClone(cropped_face, dst, src_mask, face_center, cv2.NORMAL_CLONE)
    cv2.imwrite(output_path, output)
#   plt.imshow(output) 
#   plt.show()

This is the main function for this step. Code based in the [following link](https://stackoverflow.com/a/61271319)

In [54]:
'''
Places the face from a cover into a template

In the terms of our work:

cover:  small  (just the changed aligned face)
template: big  (the original frame)
'''
def transform(template_path, cover_path, rectangle_face_coordinates, face_dimension, output_path):
    with WandImage(filename=template_path) as template:
        cover_array = cv2.imread(cover_path) #, cv2.IMREAD_UNCHANGED)
#         print("original frame...")
#         plt.imshow(cover_array)
#         plt.show()
        cover_array_resized = cv2.resize(cover_array, (template.size[1], template.size[0])) #TODO: instead of resizing the cover to the size of
#         print("resized frame..")                                                          #      the template, we can find the rectangle that
#         plt.imshow(cover_array_resized)                                                   #      the covers the tilted face
#         plt.show()
        with WandImage.from_array(cover_array_resized, channel_map="BGR") as cover:  #still not sure why we need BGR, but it works
            w, h = cover.size
            print(f"Size of cover resized: {(w, h)}")
            print(f"Size of template: {(template.size)}")
            cover.virtual_pixel = 'transparent'
            source_points = (
                  (0, 0),
                  (0, h),
                  (w, h),
                  (w, 0)
            )
            order = chain.from_iterable(zip(source_points, rectangle_face_coordinates))
            arguments = list(chain.from_iterable(order))
            print(f"arguments: {arguments}")
            cover.distort('perspective', arguments)
            ###################################
            ### Here, we crop just the face ###
            x_center = int((rectangle_face_coordinates[0][0]+ rectangle_face_coordinates[2][0])/2)
            y_center = int((rectangle_face_coordinates[0][1]+ rectangle_face_coordinates[2][1])/2)
            face_w, face_h = face_dimension
            print(f"face_dimension: {face_dimension}")
            left = x_center - (face_w // 2)
            top = y_center - (face_h // 2)
            cover.crop(left= left, top= top, width=face_w, height=face_h)
            ##################################
            # Getting te pixels of the cover 
            pixels = np.array(cover.export_pixels(channel_map= 'BGR', storage='char'), dtype="uint8").reshape((face_h, face_w, 3))
            smooth_image(template_path, pixels, face_dimension, (x_center, y_center), output_path)

Great!, now we have a way to replace an *aligned image* into our *original frame* 

Now we need to do this for every frame. However, we need the **face_coordinates** of each one of them! 

**But Raul, how are we going to get that D:**, you may ask

Fear no more my friend, as this was exactly what we had stored in *ALL_ALIGNED_INFO* from [**Step 2.1**](#Step-2.1:--We-will-get-the-aligned-images-out-of-every-frame)

### First we need to load the smile direction that's already created 

In [39]:
smile_direction = np.load('ffhq_dataset/latent_directions/smile.npy')

In [40]:
'''
Helper function. Extracts the number 1 from filenames like "frame-1.jpg"
'''
def extract_number_from_image(filename, termination = ".jpg"):
    for i in range(len(filename)): #going through every character
        char = filename[i]
        if char.isdecimal():       #we found the beginning of the number we care about!
            last_decimal_index = len(filename) - len(termination) -1
            num_extracted = filename[i:last_decimal_index + 1]
            if num_extracted.isdecimal():
                return int(num_extracted)
            else:
                raise NameError("The filename {} should be of the form [PREFIX][NUMBER][TERMINATION]".format(filename))
    raise NameError("The filename {} didnt have any decimal digits".format(filename))

In [86]:
def process_aligned_info(ALIGNED_INFO):
    for frame_name, frame_subnum, rectangle_face_coordinates in ALIGNED_INFO:
        os.makedirs(f"{general_videos_folder}{processed_aligned_frames_subfolder}{filename}/", exist_ok =True)
        frame_num = extract_number_from_image(frame_name) 
        if frame_subnum != 1:
            print(f"The frame #{frame_num} has a subnumber of {frame_subnum}, so we are skipping it coz it's probably not a picture we care about.")
        else:
            print(f"Processing frame #{frame_num}, subnum {frame_subnum}")
            original_frame_path = f"{general_videos_folder}{frames_subfolder}{filename}/frame-{frame_num}.jpg"
            news_person = np.load(f"{general_videos_folder}{latent_representations_subfolder}{filename}/frame-{frame_num}_01.npy")
            processed_aligned_frame_path = f"{general_videos_folder}{processed_aligned_frames_subfolder}{filename}/frame-{frame_num}.jpg"

            #saving the processed frame
            save_modified_image(news_person, 
                                smile_direction, 
                                1, #TODO: how to figure out? dependant on the image/video?
                                processed_aligned_frame_path)
            print("processed aligned frame saved")              
            output_path = f"{general_videos_folder}{processed_frames_subfolder}{filename}/frame-{frame_num}.jpg"
            face_dimension = (120, 250) #TODO: make this being returned in ALL_ALIGNED_INFO (need to be more general)
            print("about to transform...")
            transform(original_frame_path, processed_aligned_frame_path, rectangle_face_coordinates, face_dimension, output_path)
            print(f"transformed!, see {output_path}")
            plt.imshow(cv2.imread(output_path))
            plt.show()


Now lets try it with the information we got in [Step 2.1](#Step-2.1:--We-will-get-the-aligned-images-out-of-every-frame)

In [ ]:
process_aligned_info(ALL_ALIGNED_INFO)

# Step 3: Combining the processed frames into a video

In [87]:
def frames_to_video(filename, fps):  
    in_folder = f"{general_videos_folder}{processed_frames_subfolder}{filename}/"
    all_images = list(filter(lambda name: name.endswith(".jpg"), os.listdir(in_folder))) #only keeping the images
    all_images.sort(key = extract_number_from_image) #to order them by number, and not lexicographically ("frame-2" goes before "frame-12")
    print("Going through all frames")
    is_first = True
    for image in all_images:
        image_path = f"{in_folder}{image}"
        print(f"Image path: {image_path}")
        frame = cv2.imread(image_path)
        height, width, layers = frame.shape
        size = (width,height)
        if is_first:
            out_path = f"{general_videos_folder}{no_sound_videos_subfolder}frames_combined_{filename}.avi" #TODO: find fourcc of MP4 files
            print(f"Creating a new video file at {out_path} with fps ={fps} and size {size}")
            fourcc = cv2.VideoWriter_fourcc(*'DIVX')
            out = cv2.VideoWriter(out_path,fourcc, fps, size) #creating a video to store the frames
            is_first = False
        out.write(frame)
    out.release()
    print("Finished converting to video!")

## Now let's convert our frames to a video!

*Note 1*: As this video is created only from frames, it will not have any sound (see [next step](#Step-5:-Extract-the-audio-from-the-original-video) for this)

*Note 2*: The `video_fps` value was the one obtained in [**Step 1**](#Apply-function-to-our-video)

In [60]:
frames_to_video(filename, video_fps)

Going through all frames
Image path: ./videos/processed_frames/news_base/frame-0.jpg
Creating a new video file at ./videos/no_sound_videos/frames_combined_news_base.avi with fps =29.97002997002997 and size (1280, 720)
Image path: ./videos/processed_frames/news_base/frame-1.jpg
Image path: ./videos/processed_frames/news_base/frame-2.jpg
Image path: ./videos/processed_frames/news_base/frame-3.jpg
Image path: ./videos/processed_frames/news_base/frame-4.jpg
Image path: ./videos/processed_frames/news_base/frame-5.jpg
Image path: ./videos/processed_frames/news_base/frame-6.jpg
Image path: ./videos/processed_frames/news_base/frame-7.jpg
Image path: ./videos/processed_frames/news_base/frame-8.jpg
Image path: ./videos/processed_frames/news_base/frame-9.jpg
Image path: ./videos/processed_frames/news_base/frame-10.jpg
Image path: ./videos/processed_frames/news_base/frame-11.jpg
Image path: ./videos/processed_frames/news_base/frame-12.jpg
Image path: ./videos/processed_frames/news_base/frame-13.jp

Image path: ./videos/processed_frames/news_base/frame-136.jpg
Image path: ./videos/processed_frames/news_base/frame-137.jpg
Image path: ./videos/processed_frames/news_base/frame-138.jpg
Image path: ./videos/processed_frames/news_base/frame-139.jpg
Image path: ./videos/processed_frames/news_base/frame-140.jpg
Image path: ./videos/processed_frames/news_base/frame-141.jpg
Image path: ./videos/processed_frames/news_base/frame-142.jpg
Image path: ./videos/processed_frames/news_base/frame-143.jpg
Image path: ./videos/processed_frames/news_base/frame-144.jpg
Image path: ./videos/processed_frames/news_base/frame-145.jpg
Image path: ./videos/processed_frames/news_base/frame-146.jpg
Image path: ./videos/processed_frames/news_base/frame-147.jpg
Image path: ./videos/processed_frames/news_base/frame-148.jpg
Image path: ./videos/processed_frames/news_base/frame-149.jpg
Finished converting to video!


# Step 4: Extract the audio from the original video

In [65]:
from moviepy.editor import AudioFileClip
def extract_audio(filename):
    audio_path = f"{general_videos_folder}{inputs_subfolder}{filename}.mp4"
    audio = AudioFileClip(audio_path)
    audio.write_audiofile(f"{general_videos_folder}{audio_subfolder}audio_from_{filename}.mp3")

### Now lets try to use that on our filename

In [68]:
extract_audio(filename)

chunk:   0%|          | 0/155 [00:00<?, ?it/s, now=None]

MoviePy - Writing audio in ./videos/audios/audio_from_news_base_with_sound.mp3



chunk:   1%|          | 1/111 [05:59<10:59:42, 359.84s/it, now=None][A

MoviePy - Done.


# Step 5: Add the audio to our processed video

We are so close! Now we only need a way to combine the video we created in [`Step #3`](#Now-let's-convert-our-frames-to-a-video!) and the original audio we extracted in [`Step #4`](#Now-lets-try-to-use-that-on-our-filename)

In [70]:
from moviepy.editor import VideoFileClip
def add_audio_to_processed_video(filename):
    video_no_sound_path = f"{general_videos_folder}{no_sound_videos_subfolder}frames_combined_{filename}.avi"
    video_no_sound = VideoFileClip(video_no_sound_path)

    video_final_path = f"{general_videos_folder}{final_videos_subfolder}final_{filename}.mp4"
    audio_path = f"{general_videos_folder}{audio_subfolder}audio_from_{filename}.mp3"
    video_no_sound.write_videofile(video_final_path, audio = audio_path)

And finally ... **drumrolls**... the last step. 

In [71]:
add_audio_to_processed_video(filename)

t:   0%|          | 0/151 [00:00<?, ?it/s, now=None]

Moviepy - Building video ./videos/final_videos/final_news_base.mp4.
Moviepy - Writing video ./videos/final_videos/final_news_base.mp4




chunk:   1%|          | 1/111 [08:00<14:41:16, 480.70s/it, now=None]

Moviepy - Done !
Moviepy - video ready ./videos/final_videos/final_news_base.mp4


## See your final video!

In [108]:
from IPython.display import HTML

HTML(f"""
<div align="middle">
<video width="80%" controls>
      <source src="{general_videos_folder}{final_videos_subfolder}final_{filename}.mp4" type="video/mp4">
</video></div>""")

### And we are done, yayy!

![Good job](https://media.giphy.com/media/tctTjuqHP2Wk0/giphy.gif)

# Final thoughts:

* Still need to see a way to not harcode the face dimension
* Adding the **original** audio does not seem to be a good idea, as now the lips of the transformed frames are not in sync with the sound